<a href="https://colab.research.google.com/github/zecojls/SoilSpec4GG_GoogleColab/blob/main/SoilSpec4GG_002_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial setup and packages

1. Checking the packages-installation folder


In [ ]:
#.libPaths()

2. Checking working directory

In [ ]:
#getwd()

3. Listing working directory. *sample_data* comes from default Google Colab notebook

In [ ]:
#list.files()

4. Checking available (native) packages that are already loaded



In [ ]:
#search()

5. Checking available (native) packages that are already installed, but not loaded

In [ ]:
#with(data.frame(installed.packages()), Package)

5. Installing required packages. NOTE: the packages must be reinstalled every session

In [ ]:
install.packages("prospectr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘foreach’, ‘iterators’, ‘mathjaxr’, ‘RcppArmadillo’




6. Loading the required packages.

In [ ]:
library(tidyverse)
library(prospectr)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

prospectr version 0.2.2 -- 'flawil'

check the github repository at: http://github.com/l-ramirez-lopez/prospectr



# Loading the full OSSL dataset

Loading the full OSSL data from S3 bucket.

In [ ]:
rm.ossl = readRDS(url("http://s3.us-east-1.wasabisys.com/soilspectroscopy/ossl_import/rm.ossl_v1.rds", "rb"))
dim(rm.ossl)

[1] 152146   2962

In [ ]:
str(rm.ossl)

'data.frame':	152146 obs. of  2962 variables:
 $ id.layer_uuid_c                                             : chr  "bd934b2680a2478c8a8ce1913ae79172" "ff03863683a192953930bcb2a874b527" "1fe42f85d1cb818e99d190c80686063a" "8ea0ca98a9a8002ad7e713616d53ed86" ...
 $ id.layer_local_c                                            : chr  "icr006475" "icr006586" "icr007929" "icr008008" ...
 $ sample.doi_idf_c                                            : chr  "10.1016/j.geodrs.2015.06.002" "10.1016/j.geodrs.2015.06.002" "10.1016/j.geodrs.2015.06.002" "10.1016/j.geodrs.2015.06.002" ...
 $ sample.contact.name_utf8_txt                                : chr  "Keith Shepherd" "Keith Shepherd" "Keith Shepherd" "Keith Shepherd" ...
 $ sample.contact.email_ietf_email                             : chr  "afsis.info@africasoils.net" "afsis.info@africasoils.net" "afsis.info@africasoils.net" "afsis.info@africasoils.net" ...
 $ acid.tea_usda4b2_cmolkg                                     : num  NA NA NA NA NA NA 

Checking available SSL

In [ ]:
rm.ossl %>%
  distinct(dataset.code_ascii_c)

dataset.code_ascii_c
<chr>
AFSIS1.SSL
AFSIS2.SSL
CAF.SSL
KSSL.SSL
ICRAF.ISRIC
LUCAS.SSL
NEON.SSL
